In [40]:
import leaguepedia_parser

# Gets regions names
regions = leaguepedia_parser.get_regions()

# Gets tournaments in the region, by default only returns primary tournaments
tournaments = leaguepedia_parser.get_tournaments("Korea", year=2020)

# Gets all games for a tournament. Get the name from get_tournaments()
games = leaguepedia_parser.get_games("LCK/2021 Season/Spring Season")

# Gets picks and bans and other details from a game. Get the game object from get_games()
game = leaguepedia_parser.get_game_details(games[0])

# Gets the URL of the team’s logo
logo_url = leaguepedia_parser.get_team_logo('T1')

In [3]:
print(regions)

[None, 'Africa', 'Asia', 'Brazil', 'China', 'CIS', 'EMEA', 'Europe', 'International', 'Japan', 'Korea', 'LAN', 'LAS', 'Latin America', 'LMS', 'MENA', 'North America', 'Oceania', 'PCS', 'SEA', 'Turkey', 'Unknown', 'Vietnam', 'Wildcard']


In [4]:
print(tournaments)

[LeaguepediaTournament(name='KeSPA Cup 2019', start='2019-12-23', end='2020-01-05', region='Korea', league='KeSPA', leagueShort='KeSPA', rulebook=None, tournamentLevel='Primary', isQualifier=True, isPlayoffs=True, isOfficial=True, overviewPage='2019 LoL KeSPA Cup'), LeaguepediaTournament(name='LCK 2020 Spring Promotion', start='2019-09-09', end='2019-09-11', region='Korea', league='LoL Champions Korea', leagueShort='LCK', rulebook='http://static.leagueoflegends.co.kr/esports/files/%EB%A6%AC%EA%B7%B8%20%EC%98%A4%EB%B8%8C%20%EB%A0%88%EC%A0%84%EB%93%9C%20%EC%B1%94%ED%94%BC%EC%96%B8%EC%8A%A4%20%EC%BD%94%EB%A6%AC%EC%95%84%202019_%EA%B3%B5%EC%8B%9D%20%EA%B7%9C%EC%A0%95%EC%A7%91_(Final).pdf', tournamentLevel='Primary', isQualifier=True, isPlayoffs=True, isOfficial=True, overviewPage='LCK/2020 Season/Spring Promotion'), LeaguepediaTournament(name='LCK 2020 Spring', start='2020-02-05', end='2020-04-16', region='Korea', league='LoL Champions Korea', leagueShort='LCK', rulebook='http://static.lea

In [45]:
#print(games)
import pandas as pd
df = pd.DataFrame(games)
df.head()
#df.to_csv("trial2.csv")

print(df["teams"][1])

{'BLUE': {'bans': [79, 76, 236, 142, 126], 'players': [{'primaryRuneTreeId': None, 'secondaryRuneTreeId': None, 'championId': 150, 'id': None, 'inGameName': None, 'profileIconId': None, 'role': None, 'sources': {}, 'runes': [], 'summonerSpells': [], 'endOfGameStats': None, 'snapshots': [], 'itemsEvents': [], 'wardsEvents': [], 'skillsLevelUpEvents': [], 'largeMonstersKills': [], 'levelUpEvents': [], 'spellsUses': [], 'specialKills': []}, {'primaryRuneTreeId': None, 'secondaryRuneTreeId': None, 'championId': 80, 'id': None, 'inGameName': None, 'profileIconId': None, 'role': None, 'sources': {}, 'runes': [], 'summonerSpells': [], 'endOfGameStats': None, 'snapshots': [], 'itemsEvents': [], 'wardsEvents': [], 'skillsLevelUpEvents': [], 'largeMonstersKills': [], 'levelUpEvents': [], 'spellsUses': [], 'specialKills': []}, {'primaryRuneTreeId': None, 'secondaryRuneTreeId': None, 'championId': 112, 'id': None, 'inGameName': None, 'profileIconId': None, 'role': None, 'sources': {}, 'runes': [],

In [7]:
print(logo_url)

https://static.wikia.nocookie.net/lolesports_gamepedia_en/images/a/a2/T1logo_square.png/revision/latest?cb=20230512040747


In [8]:
import dataclasses
from typing import Optional
from leaguepedia_parser.site.leaguepedia import leaguepedia


@dataclasses.dataclass
class TeamAssets:
    thumbnail_url: str
    logo_url: str
    long_name: str  # Aka display name


def get_all_team_assets(team_link: str) -> TeamAssets:
    """

    Args:
        team_link: a field coming from Team1/Team2 in ScoreboardGames

    Returns:
        A TeamAssets object

    """
    result = leaguepedia.site.client.api(
        action="query",
        format="json",
        prop="imageinfo",
        titles=f"File:{team_link}logo square.png|File:{team_link}logo std.png",
        iiprop="url",
    )

    pages = result["query"]["pages"]

    urls = []
    for v in pages.values():
        urls.append(v["imageinfo"][0]["url"])

    long_name = leaguepedia.site.cache.get("Team", team_link, "link")

    return TeamAssets(
        thumbnail_url=urls[1],
        logo_url=urls[0],
        long_name=long_name,
    )


def get_team_logo(team_name: str, _retry=True) -> str:
    """
    Returns the team logo URL

    Params:
        team_name: Team name, usually gotten from the game dictionary
        _retry: whether or not to get the team’s full name from Leaguepedia if it was not understood

    Returns:
        URL pointing to the team’s logo
    """
    return _get_team_asset(f"File:{team_name}logo square.png", team_name, _retry)


def get_team_thumbnail(team_name: str, _retry=True) -> str:
    """
    Returns the team thumbnail URL

    Params:
        team_name: Team name, usually gotten from the game dictionary
        _retry: whether or not to get the team’s full name from Leaguepedia if it was not understood

    Returns:
        URL pointing to the team’s thumbnail
    """
    return _get_team_asset(f"File:{team_name}logo std.png", team_name, _retry)


def _get_team_asset(asset_name: str, team_name: str, _retry=True) -> str:
    """
    Returns the team thumbnail URL

    Params:
        team_name: Team name, usually gotten from the game dictionary
        _retry: whether or not to get the team’s full name from Leaguepedia if it was not understood

    Returns:
        URL pointing to the team’s logo
    """
    result = leaguepedia.site.client.api(
        action="query",
        format="json",
        prop="imageinfo",
        titles=asset_name,
        iiprop="url",
    )

    try:
        url = None
        pages = result.get("query").get("pages")
        for k, v in pages.items():
            url = v.get("imageinfo")[0].get("url")

    except (TypeError, AttributeError):
        # This happens when the team name was not properly understood.
        if _retry:
            return get_team_logo(get_long_team_name_from_trigram(team_name), False)
        else:
            raise Exception("Logo not found for the given team name")

    return url


def get_long_team_name_from_trigram(
    team_abbreviation: str,
    event_overview_page: str = None,
) -> Optional[str]:
    """
    Returns the long team name for the given team abbreviation using Leaguepedia’s search pages

    Only issues a query the first time it is called, then stores the data in a cache
    There is no cache timeout at the moment

    Args:
        team_abbreviation: A team name abbreviation, like IG or RNG
        event_overview_page: The overviewPage field of the tournament, useful for disambiguation

    Returns:
        The long team name, like "Invictus Gaming" or "Royal Never Give Up"
    """

    # We use only lowercase team abbreviations for simplicity
    team_abbreviation = team_abbreviation.lower()

    if event_overview_page:
        return leaguepedia.site.cache.get_team_from_event_tricode(
            event_overview_page, team_abbreviation
        )

    else:
        return leaguepedia.site.cache.get("Team", team_abbreviation, "link")

In [10]:
get_all_team_assets("T1")

MaximumRetriesExceeded: (<mwclient.sleep.Sleeper object at 0x0000025DA70DD3D0>, ('api', OrderedDict([('format', 'json'), ('prop', 'imageinfo'), ('titles', 'File:T1logo square.png|File:T1logo std.png'), ('iiprop', 'url'), ('continue', ''), ('meta', 'userinfo'), ('uiprop', 'blockinfo|hasmsg'), ('action', 'query')])))

In [12]:
get_long_team_name_from_trigram("GEN")

'Gen.G'

In [13]:
get_team_logo("T1")

'https://static.wikia.nocookie.net/lolesports_gamepedia_en/images/a/a2/T1logo_square.png/revision/latest?cb=20230512040747'

In [14]:
from concurrent.futures.thread import ThreadPoolExecutor
from typing import List, Optional

from lol_dto.classes.game import LolGame
from lol_dto.classes.game.lol_game import LolPickBan

from leaguepedia_parser.site.leaguepedia import leaguepedia
from leaguepedia_parser.transmuters.field_names import (
    game_fields,
    tournaments_fields,
    game_players_fields,
)
from leaguepedia_parser.transmuters.game import transmute_game
from leaguepedia_parser.transmuters.game_players import add_players
from leaguepedia_parser.transmuters.picks_bans import (
    picks_bans_fields,
    transmute_picks_bans,
)
from leaguepedia_parser.transmuters.tournament import (
    transmute_tournament,
    LeaguepediaTournament,
)


def get_regions() -> List[str]:
    """Returns a list of all regions that appear in the Tournaments table.

    Returns:
        The list of all region names, simply strings.
    """
    regions_dicts_list = leaguepedia.query(
        tables="Tournaments", fields="Region", group_by="Region"
    )

    return [row["Region"] for row in regions_dicts_list]


def get_tournaments(
    region: str = None,
    year: int = None,
    tournament_level: str = "Primary",
    is_playoffs: bool = None,
    **kwargs,
) -> List[LeaguepediaTournament]:
    """Returns a list of tournaments.

    Typical usage example:
        get_tournaments('China', 2020)

    Args:
        region: Recommended to get it from get_tournament_regions().
        year: Year to filter on. Defaults to None.
        tournament_level: Primary, Secondary, Major, Secondary, Showmatch. Defaults to Primary.
        is_playoffs: Can be used to filter between playoffs and regular season tournaments.

    Returns:
        A list of tournaments dictionaries.
    """
    # We need to cast is_playoffs as an integer for the cargoquery
    if is_playoffs is not None:
        is_playoffs = 1 if is_playoffs else 0

    # This generates the WHERE part of the cargoquery
    where = " AND ".join(
        [
            # One constraint per variable
            f"Tournaments.{field_name}='{value}'"
            for field_name, value in [
                ("Region", region),
                ("Year", year),
                ("TournamentLevel", tournament_level),
                ("IsPlayoffs", is_playoffs),
            ]
            # We don’t filter on variables that are None
            if value is not None
        ]
    )

    result = leaguepedia.query(
        tables="Tournaments, Leagues",
        join_on="Tournaments.League = Leagues.League",
        fields=f"Leagues.League_Short, {', '.join(f'Tournaments.{field}' for field in tournaments_fields)}",
        where=where,
        **kwargs,
    )

    return [transmute_tournament(tournament) for tournament in result]


def get_games(tournament_overview_page=None, **kwargs) -> List[LolGame]:
    """Returns the list of games played in a tournament.

    Returns basic information about all games played in a tournament.

    Args:
        tournament_overview_page: tournament overview page, acquired from get_tournaments().

    Returns:
        A list of LolGame with basic game information.
    """

    games = leaguepedia.query(
        tables="ScoreboardGames",
        fields=", ".join(game_fields),
        where=f"ScoreboardGames.OverviewPage ='{tournament_overview_page}'",
        order_by="ScoreboardGames.DateTime_UTC",
        **kwargs,
    )

    return [transmute_game(game) for game in games]


def get_game_details(game: LolGame, add_page_id=False) -> LolGame:
    # TODO Add more scoreboard information in this step
    """Gets most game information available on Leaguepedia.

    Args:
        game: A LolGame with Leaguepedia IDs in its 'sources' dict.
        add_page_id: whether or not to link the player page ID to their object. Mostly for debugging.

    Returns:
        The LolGame with all information available on Leaguepedia.
    """
    try:
        assert game.sources.leaguepedia.gameId
    except AssertionError:
        raise ValueError(
            f"Leaguepedia GameId not present in the input object, joins cannot be performed to get details"
        )

    with ThreadPoolExecutor() as executor:
        picks_bans_future = executor.submit(_get_picks_bans, game)
        game_future = executor.submit(_add_game_players, game, add_page_id)

    game = game_future.result()
    game.picksBans = picks_bans_future.result()

    return game


def _get_picks_bans(game: LolGame) -> Optional[List[LolPickBan]]:
    """Returns the picks and bans for the game."""
    # Double join as required by Leaguepedia
    picks_bans = leaguepedia.query(
        tables="PicksAndBansS7, ScoreboardGames",
        join_on="PicksAndBansS7.GameId = ScoreboardGames.GameId",
        fields=", ".join(picks_bans_fields),
        where=f"ScoreboardGames.GameId = '{game.sources.leaguepedia.gameId}'",
    )

    if not picks_bans:
        return None

    return transmute_picks_bans(picks_bans[0])


def _add_game_players(game: LolGame, add_page_id: bool) -> LolGame:
    """Joins on PlayersRedirect to get all players information."""

    players = leaguepedia.query(
        tables=f"ScoreboardGames, ScoreboardPlayers, PlayerRedirects, Players",
        join_on="ScoreboardGames.GameId = ScoreboardPlayers.GameId, "
        "ScoreboardPlayers.Link = PlayerRedirects.AllName, "
        "PlayerRedirects.OverviewPage = Players.OverviewPage",
        fields=", ".join(game_players_fields) + ", Players._pageID=pageId",
        where=f"ScoreboardGames.GameId = '{game.sources.leaguepedia.gameId}'",
    )

    return add_players(game, players, add_page_id=add_page_id)

In [16]:
get_regions()

[None,
 'Africa',
 'Asia',
 'Brazil',
 'China',
 'CIS',
 'EMEA',
 'Europe',
 'International',
 'Japan',
 'Korea',
 'LAN',
 'LAS',
 'Latin America',
 'LMS',
 'MENA',
 'North America',
 'Oceania',
 'PCS',
 'SEA',
 'Turkey',
 'Unknown',
 'Vietnam',
 'Wildcard']

In [18]:
get_tournaments(
    region = "Korea",
    year = 2022,
    tournament_level = "Primary",
    is_playoffs = True
)

[LeaguepediaTournament(name='LCK 2022 Spring Playoffs', start='2022-03-23', end='2022-04-02', region='Korea', league='LoL Champions Korea', leagueShort='LCK', rulebook='Archive:Official Rulebooks/Riot/LCK/2022', tournamentLevel='Primary', isQualifier=True, isPlayoffs=True, isOfficial=True, overviewPage='LCK/2022 Season/Spring Playoffs'),
 LeaguepediaTournament(name='LCK 2022 Summer Playoffs', start='2022-08-17', end='2022-08-28', region='Korea', league='LoL Champions Korea', leagueShort='LCK', rulebook='https://assets.contentstack.io/v3/assets/bltad9188aa9a70543a/bltf131591935967df7/62a3597d03b23c505f83f03a/2022_LCK_Summer_%EA%B7%9C%EC%A0%95%EC%A7%91.pdf', tournamentLevel='Primary', isQualifier=True, isPlayoffs=True, isOfficial=True, overviewPage='LCK/2022 Season/Summer Playoffs')]

In [36]:
from concurrent.futures.thread import ThreadPoolExecutor
from typing import List, Optional

from lol_dto.classes.game import LolGame
from lol_dto.classes.game.lol_game import LolPickBan

from leaguepedia_parser.site.leaguepedia import leaguepedia
from leaguepedia_parser.transmuters.field_names import (
    game_fields,
    tournaments_fields,
    game_players_fields,
)
from leaguepedia_parser.transmuters.game import transmute_game
from leaguepedia_parser.transmuters.game_players import add_players
from leaguepedia_parser.transmuters.picks_bans import (
    picks_bans_fields,
    transmute_picks_bans,
)
from leaguepedia_parser.transmuters.tournament import (
    transmute_tournament,
    LeaguepediaTournament,
)


def get_regions() -> List[str]:
    """Returns a list of all regions that appear in the Tournaments table.

    Returns:
        The list of all region names, simply strings.
    """
    regions_dicts_list = leaguepedia.query(
        tables="Tournaments", fields="Region", group_by="Region"
    )

    return [row["Region"] for row in regions_dicts_list]


def get_tournaments(
    region: str = None,
    year: int = None,
    tournament_level: str = "Primary",
    is_playoffs: bool = None,
    **kwargs,
) -> List[LeaguepediaTournament]:
    """Returns a list of tournaments.

    Typical usage example:
        get_tournaments('China', 2020)

    Args:
        region: Recommended to get it from get_tournament_regions().
        year: Year to filter on. Defaults to None.
        tournament_level: Primary, Secondary, Major, Secondary, Showmatch. Defaults to Primary.
        is_playoffs: Can be used to filter between playoffs and regular season tournaments.

    Returns:
        A list of tournaments dictionaries.
    """
    # We need to cast is_playoffs as an integer for the cargoquery
    if is_playoffs is not None:
        is_playoffs = 1 if is_playoffs else 0

    # This generates the WHERE part of the cargoquery
    where = " AND ".join(
        [
            # One constraint per variable
            f"Tournaments.{field_name}='{value}'"
            for field_name, value in [
                ("Region", region),
                ("Year", year),
                ("TournamentLevel", tournament_level),
                ("IsPlayoffs", is_playoffs),
            ]
            # We don’t filter on variables that are None
            if value is not None
        ]
    )

    result = leaguepedia.query(
        tables="Tournaments, Leagues",
        join_on="Tournaments.League = Leagues.League",
        fields=f"Leagues.League_Short, {', '.join(f'Tournaments.{field}' for field in tournaments_fields)}",
        where=where,
        **kwargs,
    )

    return [transmute_tournament(tournament) for tournament in result]


def get_games(tournament_overview_page=None, **kwargs) -> List[LolGame]:
    """Returns the list of games played in a tournament.

    Returns basic information about all games played in a tournament.

    Args:
        tournament_overview_page: tournament overview page, acquired from get_tournaments().

    Returns:
        A list of LolGame with basic game information.
    """

    games = leaguepedia.query(
        tables="ScoreboardGames",
        fields=", ".join(game_fields),
        where=f"ScoreboardGames.OverviewPage ='{tournament_overview_page}'",
        order_by="ScoreboardGames.DateTime_UTC",
        **kwargs,
    )

    return [transmute_game(game) for game in games]


def get_game_details(game: LolGame, add_page_id=False) -> LolGame:
    # TODO Add more scoreboard information in this step
    """Gets most game information available on Leaguepedia.

    Args:
        game: A LolGame with Leaguepedia IDs in its 'sources' dict.
        add_page_id: whether or not to link the player page ID to their object. Mostly for debugging.

    Returns:
        The LolGame with all information available on Leaguepedia.
    """
    try:
        assert game.sources.leaguepedia.gameId
    except AssertionError:
        raise ValueError(
            f"Leaguepedia GameId not present in the input object, joins cannot be performed to get details"
        )

    with ThreadPoolExecutor() as executor:
        picks_bans_future = executor.submit(_get_picks_bans, game)
        game_future = executor.submit(_add_game_players, game, add_page_id)

    game = game_future.result()
    game.picksBans = picks_bans_future.result()

    return game


def _get_picks_bans(game: LolGame) -> Optional[List[LolPickBan]]:
    """Returns the picks and bans for the game."""
    # Double join as required by Leaguepedia
    picks_bans = leaguepedia.query(
        tables="PicksAndBansS7, ScoreboardGames",
        join_on="PicksAndBansS7.GameId = ScoreboardGames.GameId",
        fields=", ".join(picks_bans_fields),
        where=f"ScoreboardGames.GameId = '{game.sources.leaguepedia.gameId}'",
    )

    if not picks_bans:
        return None

    return transmute_picks_bans(picks_bans[0])


def _add_game_players(game: LolGame, add_page_id: bool) -> LolGame:
    """Joins on PlayersRedirect to get all players information."""

    players = leaguepedia.query(
        tables=f"ScoreboardGames, ScoreboardPlayers, PlayerRedirects, Players",
        join_on="ScoreboardGames.GameId = ScoreboardPlayers.GameId, "
        "ScoreboardPlayers.Link = PlayerRedirects.AllName, "
        "PlayerRedirects.OverviewPage = Players.OverviewPage",
        fields=", ".join(game_players_fields) + ", Players._pageID=pageId",
        where=f"ScoreboardGames.GameId = '{game.sources.leaguepedia.gameId}'",
    )

    return add_players(game, players, add_page_id=add_page_id)

In [38]:
get_games("LCK/2022 Season/Summer Season")

MaximumRetriesExceeded: (<mwclient.sleep.Sleeper object at 0x0000025DAC7EDD00>, ('api', OrderedDict([('tables', 'ScoreboardGames'), ('fields', 'Team2Towers, Tournament, Team2Players, Team1Players, Team2Dragons, Team2Bans, Gamelength_Number, VOD, Team1Barons, Team2RiftHeralds, GameId, Team1Score, Team1Picks, Patch, Team2Barons, Team1Towers, Team2Inhibitors, Team1Bans, DateTime_UTC, Team1Inhibitors, N_GameInMatch, MatchId, Team1, Gamename, Team2, Winner, MatchHistory, OverviewPage, Team2Picks, RiotPlatformGameId, Team1Dragons, Team1RiftHeralds, Team2Score'), ('where', "ScoreboardGames.OverviewPage ='LCK/2022 Season/Summer Season'"), ('order_by', 'ScoreboardGames.DateTime_UTC'), ('offset', 0), ('limit', 500), ('action', 'cargoquery'), ('format', 'json')])))